# Execution of tasks on remote hosts, an overview

* **Difficulty level**: intermediate
* **Time need to lean**: 15 minutes or less
* **Key points**:
  * A configuration file specifies how SoS should interact with a remote host
  * Tasks should be written in a way that can be executed remotely
  * Tasks can be submitted to remote hosts with option `-q`

This tutorial gives an overview of the SoS task model and steps to submit tasks to remote hosts.

## How remote task execution works

Executing tasks on remote hosts can be very tricky so it is important to understand how SoS works, what SoS provides and what are the limitations. Here is a summary of how SoS execute tasks on remote hosts:

**Step 1: Prepare task**
* SoS generates a task file, which contains information such as the content of the task (e.g. script to execute) and environments (variables, parameter etc).
* If the remote system is a batch system such as `PBS`, `SLURM` and `LSF`, a shell script is generated from a host-specific template. 

**Step 2: Copying tasks and reuired files to remote host**
* SoS prepares the task file to be executed on the remote host. It essentially records the path definitions of remote host so that the task can be executed with correct paths.
* SoS copies the task file, shell script, and sometimes input files to remote host using `rsync` or `rcp`.

**Step 3: Execute task remotely, optionally througgh a task queue**
* If the remote system is not a batch system, a `sos execute task_id` is started on the remote host (through `ssh`) to execute the task.
* If the remote system is a batch system, a system-dependent command (e.g. `qsub task_id.sh`) is executed on the remote host to submit the task to the scheduler. The shell script is essentially a wrapper to the `sos execute` command.

**Step 4: Monitor the execute of jobs**
* The task is run on remote host, either directly or by a batch system.
* SoS periodically sends a `sos status` command to remote host to check the status of the task.

**Step 5: Retrieve its result once it is done**
* Once the task is completed, SoS copies the task file and specified output file back to local host.

<p align="center">
  <img src="https://vatlab.github.io/sos-docs/doc/media/task_overview.png">
</p>

Let us walk through the setup and execution of this process with an example.

## System configuration

If you think through the SoS task model, 

|Public-key access to remote hosts and tools such as `ssh`, `rsync` and `scp` | Unlike other programs that require a deamon process (a service) on the server to coordinate the execution of tasks, SoS relies on a basic remote access toolchain to access the remote host and execute commands remotely. 

| `sos remote setup` |
|`sos` and required programs on remote host | The `sos` command is required on the remote host to execute the task with appropriate interpreters and tools | `sos remote test` |

## Further reading

* 